In [1]:
import sys  
sys.path.insert(0, '/content/drive/MyDrive/overoptimism')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install optuna

     |████████████████████████████████| 296kB 11.8MB/s 
     |████████████████████████████████| 1.2MB 20.5MB/s 
     |████████████████████████████████| 81kB 5.9MB/s 
     |████████████████████████████████| 81kB 6.4MB/s 
     |████████████████████████████████| 143kB 49.6MB/s 
     |████████████████████████████████| 51kB 3.5MB/s 
     |████████████████████████████████| 112kB 53.7MB/s 
  Created wheel for alembic: filename=alembic-1.6.2-py2.py3-none-any.whl size=164219 sha256=e418098d1c12d9a8764731a3ad3d8c2318376eea2e355dfcef65c503642babff
  Stored in directory: /root/.cache/pip/wheels/a5/63/44/db29401e49ef5331c163b591f12a465c40af864bfa888dabd2
  Created wheel for pyperclip: filename=pyperclip-1.8.2-cp37-none-any.whl size=11107 sha256=a9ef3604ad0755a6bf6130285a867a1f60f8a7e84ccc322842265beb2e8590b3
  Stored in directory: /root/.cache/pip/wheels/25/af/b8/3407109267803f4015e1ee2ff23be0c8c19ce4008665931ee1
Successfully built alembic pyperclip


In [4]:
%load_ext autoreload
%autoreload 2

In [7]:
import optuna
from optuna.samplers import TPESampler
import numpy as np
import pandas as pd

from sklearn.datasets import make_blobs, make_moons
from sklearn.cluster import KMeans, DBSCAN, SpectralClustering, MeanShift, estimate_bandwidth
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import adjusted_mutual_info_score as AMI
from rock import ROCK

from util import dbscan_init

import logging
import sys

In [15]:
def getBlobDensities(n_centers):
  return np.round([3/n_centers * x for x in range(1, n_centers + 1)], decimals=2)

In [14]:
def objective(trial):
    seed = 0

    rock_results = []
    other_results = []

    n_samples = trial.suggest_int('n_samples', 1, 16) * 100
    n_features = trial.suggest_int('n_features', 2, 20)
    n_centers = trial.suggest_int('n_centers', 2, 10)

    datasets = []

    for s in range(seed, seed+10):
        datasets.append(make_blobs(n_samples=n_samples, centers=n_centers, n_features=n_features, cluster_std=getBlobDensities(n_centers), random_state=s))
        
    step = 0
    for dataset in datasets: 
        run = {}
        run['run_id'] = run_id
        run['step'] = step
        run['n_samples'] = n_samples
        run['n_centers'] = n_centers
        run['n_features'] = n_features
        scaler = StandardScaler()
        data = scaler.fit_transform(dataset[0])
        kmeans = KMeans(n_clusters=n_centers, random_state=seed).fit(data).labels_
        run['k'] = n_centers
        eps, min_pts = (0.2, 2 * n_features)
        dbscan = DBSCAN(eps=eps, min_samples=min_pts).fit(data).labels_
        run['eps'] = eps
        run['min_pts'] = 2 * n_features
        spectral = SpectralClustering(n_clusters=n_centers).fit(data).labels_
        bandwidth = estimate_bandwidth(data)
        run['bandwidth'] = bandwidth
        mean_shift = MeanShift(bandwidth=bandwidth).fit(data).labels_

        rock = ROCK(tmax=15).fit(data).labels_

        gt = dataset[1]
        rock_results.append(AMI(rock, gt))
        other_results.append(np.max([AMI(kmeans, gt), AMI(dbscan, gt), AMI(spectral, gt), AMI(mean_shift, gt)]))

        run['ROCK'] = AMI(rock, gt)
        run['K_MEANS'] = AMI(kmeans, gt)
        run['DBSCAN'] = AMI(dbscan, gt)
        run['SPECTRAL'] = AMI(spectral, gt)
        run['MEAN_SHIFT'] = AMI(mean_shift, gt)

        step += 1

        experiment.append(run)

    return np.mean(rock_results) - np.mean(other_results)

In [16]:
experiment = []
run_id = 0

In [17]:
study_name = 'den_blobs'

In [18]:
storage_name = "sqlite:///{}.db".format(study_name)
sampler = TPESampler(seed=42)
study = optuna.create_study(study_name=study_name, storage=storage_name, sampler=sampler, load_if_exists=True,  direction="maximize")

[I 2021-05-15 11:25:22,913] A new study created in RDB with name: den_blobs


In [19]:
study.optimize(objective, n_trials=100)

[I 2021-05-15 11:25:59,629] Trial 0 finished with value: -0.6082313190701352 and parameters: {'n_samples': 6, 'n_features': 20, 'n_centers': 8}. Best is trial 0 with value: -0.6082313190701352.
[I 2021-05-15 11:26:52,746] Trial 1 finished with value: -0.03773364939895263 and parameters: {'n_samples': 10, 'n_features': 4, 'n_centers': 3}. Best is trial 1 with value: -0.03773364939895263.
[I 2021-05-15 11:27:12,670] Trial 2 finished with value: -0.8526323451498099 and parameters: {'n_samples': 1, 'n_features': 18, 'n_centers': 7}. Best is trial 1 with value: -0.03773364939895263.
[I 2021-05-15 11:28:33,205] Trial 3 finished with value: -0.21641025190500496 and parameters: {'n_samples': 12, 'n_features': 2, 'n_centers': 10}. Best is trial 1 with value: -0.03773364939895263.
[I 2021-05-15 11:29:31,774] Trial 4 finished with value: -0.16328197707720915 and parameters: {'n_samples': 14, 'n_features': 6, 'n_centers': 3}. Best is trial 1 with value: -0.03773364939895263.
[I 2021-05-15 11:29:59

In [21]:
pd.DataFrame(experiment).to_csv('den_blobs.csv')